In [1]:
data = r"C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\ALL FILE CSV\all_file.csv"
stopword_id = r'C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\stopwords-id.txt'

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import re
from joblib import Parallel, delayed
from joblib import Memory

# Caching setup
location = './cache'
memory = Memory(location, verbose=0)

# Inisialisasi stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membaca daftar stop words bahasa Indonesia
with open(stopword_id, 'r') as f:
    stop_words_id = f.read().splitlines()

def preprocess_text(text):
    # Pastikan text adalah string
    if not isinstance(text, str):
        text = str(text)
    
    # Menghilangkan karakter berulang
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Menghilangkan angka
    text = ''.join([i for i in text if not i.isdigit()])
    
    # Menghilangkan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    
    # Melakukan stemming pada teks
    text = stemmer.stem(text)
    
    return text

@memory.cache
def preprocess_parallel(text_series):
    return Parallel(n_jobs=-1)(delayed(preprocess_text)(text) for text in text_series)

# Membaca file CSV
df = pd.read_csv(data)

# Pastikan tidak ada nilai NaN dan semua nilai adalah string
df['Teks'] = preprocess_parallel(df['Teks'].fillna(''))

# Inisialisasi TfidfVectorizer dengan stop words bahasa Indonesia
tfidf = TfidfVectorizer(stop_words=stop_words_id, max_df=0.85, min_df=2, ngram_range=(1, 2))

# Melakukan fit dan transformasi pada kolom Teks
tfidf_matrix = tfidf.fit_transform(df['Teks'])

def search_documents(query, top_n=10):
    # Preprocessing query
    query = preprocess_text(query)
    
    # Transformasi query menjadi vektor tf-idf
    query_vec = tfidf.transform([query])
    
    # Menghitung cosine similarity antara query dan semua dokumen
    cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
    
    # Mendapatkan indeks dokumen dengan similarity tertinggi
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Mendapatkan judul, teks, dan nilai similarity dari dokumen yang relevan
    results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i], i) for i in related_docs_indices]
    
    return results

# Contoh penggunaan
if __name__ == "__main__":
    query = "pidana pencurian"
    results = search_documents(query)
    for title, text, similarity, doc_index in results:
        print(f"Judul: {title}")
        print(f"Similarity: {similarity:.4f}")
        print()

Judul: Undang-Undang Nomor 1 Tahun 2023
Similarity: 0.3010

Judul: Undang-Undang Nomor 31 Tahun 1999
Similarity: 0.1694

Judul: Undang-Undang Nomor 20 Tahun 2001
Similarity: 0.1491

Judul: Undang-Undang Nomor 21 Tahun 2007
Similarity: 0.1349

Judul: Undang-Undang Nomor 4 Tahun 1976
Similarity: 0.1106

Judul: Undang-Undang Nomor 11 Tahun 1980
Similarity: 0.1008

Judul: Undang-Undang Nomor 5 Tahun 2018
Similarity: 0.0993

Judul: Undang-Undang Nomor 27 Tahun 1999
Similarity: 0.0945

Judul: Undang-Undang Nomor 3 Tahun 1971
Similarity: 0.0935

Judul: Undang-Undang Nomor 4 Tahun 1958
Similarity: 0.0910



In [8]:
import numpy as np
from sklearn.metrics.pairwise import linear_kernel

def get_feedback(results):
    feedback = []
    for idx, (title, text, similarity, doc_index) in enumerate(results):
        print(f"Dokumen {idx + 1}:")
        print(f"Judul: {title}")
        print(f"Teks: {text[:200]}...")  # Display only the first 200 characters
        print(f"Similarity: {similarity:.4f}")
        relevansi = int(input("Masukkan nilai relevansi (1-5): "))
        feedback.append((doc_index, relevansi))
    return feedback

def optimize_with_feedback(feedback, tfidf_matrix):
    relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
    non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
    
    if not relevant_docs:
        print("Tidak ada dokumen yang dianggap relevan. Pencarian ulang tidak dapat dilakukan.")
        return None
    
    relevant_matrix = tfidf_matrix[relevant_docs]
    non_relevant_matrix = tfidf_matrix[non_relevant_docs] if non_relevant_docs else np.zeros(relevant_matrix.shape)
    
    # Compute the centroid of relevant and non-relevant documents
    relevant_centroid = np.asarray(relevant_matrix.mean(axis=0)).flatten()
    non_relevant_centroid = np.asarray(non_relevant_matrix.mean(axis=0)).flatten() if non_relevant_docs else np.zeros(relevant_centroid.shape)
    
    # Update query vector by moving it towards the relevant centroid and away from the non-relevant centroid
    def adjust_query_vec(query_vec, relevant_centroid, non_relevant_centroid, alpha=1, beta=0.75, gamma=0.15):
        return alpha * query_vec + beta * relevant_centroid - gamma * non_relevant_centroid
    
    return adjust_query_vec

# Example usage
if __name__ == "__main__":
    query = "pidana pencurian"
    initial_results = search_documents(query)
    
    feedback = get_feedback(initial_results)
    adjust_query_vec = optimize_with_feedback(feedback, tfidf_matrix)
    
    if adjust_query_vec:
        relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
        non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
        
        # Reprocess the query with the adjusted query vector
        query_vec = tfidf.transform([preprocess_text(query)])
        adjusted_query_vec = adjust_query_vec(query_vec.toarray(), np.asarray(tfidf_matrix[relevant_docs].mean(axis=0)).flatten(), 
                                              np.asarray(tfidf_matrix[non_relevant_docs].mean(axis=0)).flatten() if non_relevant_docs else np.zeros(query_vec.shape))
        
        # Compute cosine similarity with the adjusted query vector
        cosine_similarities = linear_kernel(adjusted_query_vec, tfidf_matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[-10:][::-1]
        
        # Display optimized results
        optimized_results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
        print("\n\n -- HASIL PENELUSURAN ULANG -- \n\n")
        for title, text, similarity in optimized_results:
            print(f"Judul: {title}")
            print(f"Similarity: {similarity:.4f}")
            print()

Dokumen 1:
Judul: Undang-Undang Nomor 1 Tahun 2023
Teks: salinanmenimbangfresidenrepubuk indonesiaundangundang republik indonesianomor tahun tentangkitab undangundang hukum pidanadengan rahmat tuhan yang maha esapresiden republik indonesiaabahwa untuk wujud...
Similarity: 0.3010
Dokumen 2:
Judul: Undang-Undang Nomor 31 Tahun 1999
Teks: presiden republik indonesia undangundang republik indonesia nomor tahun tentang berantas tindak pidana korupsi dengan rahmat tuhan yang maha esa presiden republik indonesia timbang a bahwa tindak pida...
Similarity: 0.1694
Dokumen 3:
Judul: Undang-Undang Nomor 20 Tahun 2001
Teks: presidenrepublik indonesiaundang undang republik indones ianomor tahun tentangperubahan atas undang undang nomor tahun tentang berantas tindak pidana korupsidengan rahmat tuhan yang maha esapresiden r...
Similarity: 0.1491
Dokumen 4:
Judul: Undang-Undang Nomor 21 Tahun 2007
Teks: undangundang republik indonesianomor tahun tentangpemberantasan tindak pidana dagang orangdengan rah